# Discounting of Probabilistic Food Reinforcement by Pigeons

This notebook replicates the analyses from the publication:

> Oliveira, L., Green, L., Myerson, J., & Wan, H. (2025). Discounting of probabilistic food reinforcement by pigeons. *Journal of the Experimental Analysis of Behavior, 124*(1). https://doi.org/10.1002/jeab.70042

The original R analysis is translated into a Python workflow using `pandas`, `pymc`, `arviz`, and `matplotlib`. The analysis uses Bayesian nonlinear multilevel models to fit a hyperboloid discounting function to pigeons' choices. **Experiment 1** examines choices between a certain and a probabilistic reward. **Experiment 2** extends this to choices between two probabilistic rewards.

The data for this study are publicly available on the Open Science Framework at: <https://osf.io/scwg3/>.

In [ ]:
# Install necessary packages
import sys
!{sys.executable} -m pip install pandas numpy pymc arviz matplotlib seaborn scikit-learn openpyxl statsmodels

In [3]:
# --- Setup: Imports and Custom Functions ---
import pandas as pd
import numpy as np
import warnings
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import auc as calculate_auc
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.3f}'.format

def set_mattheme(ax, title=""):
    """Applies a consistent plot theme."""
    ax.set_title(title, fontsize=16, fontweight='bold')
    ax.set_xlabel(ax.get_xlabel(), fontsize=14, fontweight='bold')
    ax.set_ylabel(ax.get_ylabel(), fontsize=14, fontweight='bold')
    ax.tick_params(axis='both', which='major', labelsize=10, direction='out')
    for spine in ax.spines.values():
        spine.set_edgecolor('black')
        spine.set_linewidth(1)
    ax.set_facecolor('white')
    ax.grid(False)

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


In [4]:
# --- Data Loading and Processing ---
# Load OSF Data
rawdat_exp1 = pd.read_csv("r code/OSF/PigeonPD_Exp1.csv")
rawdat_exp2 = pd.read_csv("r code/OSF/PigeonPD_Exp2.csv")

# Process Experiment 1 Data
disc_dat_exp1 = rawdat_exp1.copy()
disc_dat_exp1['Experiment'] = 1
disc_dat_exp1['Probability'] = disc_dat_exp1['Probability'].astype(str)
disc_dat_exp1['Rep'] = disc_dat_exp1['Probability'].str.contains('R').astype(int)
disc_dat_exp1['Prob'] = disc_dat_exp1['Probability'].str.replace('R', '').astype(float) / 100
disc_dat_exp1['odds'] = (1 - disc_dat_exp1['Prob']) / disc_dat_exp1['Prob']
disc_dat_exp1['rsv'] = disc_dat_exp1['SV'] / disc_dat_exp1['Amount']
disc_dat_exp1['Multiplier'] = 1.0
disc_dat_exp1['odds_m'] = 0

# Process Experiment 2 Data
disc_dat_exp2 = rawdat_exp2.copy()
disc_dat_exp2['Experiment'] = 2
disc_dat_exp2['Multiplier'] = disc_dat_exp2['Multiplier'].astype(str)
disc_dat_exp2['Rep'] = disc_dat_exp2['Multiplier'].str.contains('R').astype(int)
disc_dat_exp2['Multiplier'] = disc_dat_exp2['Multiplier'].str.replace('R', '').astype(float)
disc_dat_exp2['Prob_Lg'] = disc_dat_exp2['Probability_Lg'] / 100
disc_dat_exp2['Prob_Sm'] = disc_dat_exp2['Probability_Sm'] / 100
disc_dat_exp2['odds'] = (1 - disc_dat_exp2['Prob_Lg']) / disc_dat_exp2['Prob_Lg']
disc_dat_exp2['odds_m'] = (1 - disc_dat_exp2['Prob_Sm']) / disc_dat_exp2['Prob_Sm']
disc_dat_exp2['rsv'] = disc_dat_exp2['SV'] / disc_dat_exp2['Amount']
disc_dat_exp2.rename(columns={'Prob_Lg': 'Prob'}, inplace=True)

# Combine Data and Create Mean Subject
# CORRECTED: Added 'Subject' to the list of columns to keep.
cols = ['Subject', 'Experiment', 'Multiplier', 'Amount', 'Rep', 'odds', 'rsv', 'odds_m', 'Prob']
disc_dat = pd.concat([disc_dat_exp1[cols], disc_dat_exp2[cols]])

disc_mean = disc_dat[disc_dat['Rep'] == 0].groupby(
    ['Experiment', 'Amount', 'Multiplier', 'odds', 'Prob', 'odds_m']
).agg(rsv=('rsv', 'mean')).reset_index()
disc_mean['Subject'] = "Mean"
disc_mean['Rep'] = 0

disc_dat = pd.concat([disc_dat, disc_mean])
subject_levels = ["P41","P42","P43","P44","P45","P46","P47","P48","Mean"]
disc_dat['Subject'] = pd.Categorical(disc_dat['Subject'], categories=subject_levels, ordered=True)

# Create AUC data
auc_dat = disc_dat[disc_dat['Rep'] == 0].sort_values('odds').groupby(
    ['Subject', 'Experiment', 'Multiplier', 'Amount']
).apply(
    lambda g: calculate_auc(g['odds'] / g['odds'].max(), g['rsv'])
).reset_index(name='auc')

print("Data processing complete.")

Data processing complete.


---
## Experiment 1: Certain vs. Probabilistic Rewards

In Experiment 1, pigeons chose between a smaller, certain reinforcer and a larger, probabilistic reinforcer. The following section fits a hyperboloid discounting model to the data for each amount (16 and 32 pellets).

In [ ]:
# --- Experiment 1: Bayesian Model Fitting ---
def fit_exp1_model(data_filter):
    coords = {"Subject": data_filter['Subject'].cat.categories}
    with pm.Model(coords=coords) as model:
        # Data
        subject_idx = pd.Categorical(data_filter['Subject'], categories=coords["Subject"]).codes
        odds_data = pm.Data("odds_data", data_filter['odds'].values)
        
        # Priors
        h = pm.TruncatedNormal("h", mu=1, sigma=10, lower=0, dims="Subject")
        s = pm.TruncatedNormal("s", mu=1, sigma=10, lower=0, dims="Subject")
        nu = pm.HalfCauchy("nu", beta=10, dims="Subject")

        # CORRECTED: Define the model on the logit scale, matching the brms formula
        p = 1 / (1 + h[subject_idx] * odds_data)**s[subject_idx]
        # eta is the linear predictor on the logit scale
        eta = pm.math.log(p / (1 - p))
        # mu is the mean, which is the inverse-logit of eta
        mu = pm.math.invlogit(eta)
        
        # Likelihood
        pm.Beta("rsv", mu=mu, nu=nu[subject_idx], observed=data_filter['rsv'].values)
        
        # Sampling
        idata = pm.sample(draws=2000, tune=2000, chains=4, cores=4, random_seed=42,
                          progressbar=False, target_accept=0.95)
    return idata

# Fit models for both amounts
idata_exp1_16 = fit_exp1_model(disc_dat[(disc_dat.Experiment == 1) & (disc_dat.Rep == 0) & (disc_dat.Amount == 16)])
idata_exp1_32 = fit_exp1_model(disc_dat[(disc_dat.Experiment == 1) & (disc_dat.Rep == 0) & (disc_dat.Amount == 32)])

print("Experiment 1 model fitting complete.")

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [h, s, nu]
